In [1]:
import numpy as np 
import pandas as pd 
import nltk 
import string
import re
stop_words = nltk.corpus.stopwords.words('english')
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
#import matplotlib.pyplot as plt
#%matplotlib inline
from gensim.models import KeyedVectors
import spacy
import wget
import urllib.request
import tensorflow as tf
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.layers.embeddings import Embedding
import contractions
from tensorflow.keras.layers import Dropout
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.layers import Dense,Activation,LSTM,Embedding
from nltk.stem import WordNetLemmatizer
lemmatizer =  WordNetLemmatizer()
from nltk.corpus import wordnet
#from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D



In [2]:
train = pd.read_csv(r"C:\Users\Thiyaneshwaran\hackathon_NLP\train.csv")
test = pd.read_csv(r"C:\Users\Thiyaneshwaran\hackathon_NLP\test.csv")
train['source'] = 'train'
test['source'] = 'test'


In [8]:
train.shape,test.shape

((7920, 4), (1953, 3))

In [3]:
train_tweet = train.tweet
test_tweet = test.tweet

In [4]:
combi = train_tweet.append(test_tweet,ignore_index=True)


C:\Users\Thiyaneshwaran\AppData\Local\Temp\ipykernel_17608\1549643125.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combi = train_tweet.append(test_tweet,ignore_index=True)


In [9]:
combi.shape

(9873,)

In [5]:
# Pre Processing 
from bs4 import BeautifulSoup
import tqdm 
import re 
import unicodedata
import string
import contractions

def strip_html_tags(text):
    soup = BeautifulSoup(text,'html.parser')
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text



def pre_process_corpus(doc):
    norm_corpus =[]
    for text in tqdm.tqdm(doc):
        text = strip_html_tags(text)
        text = remove_accented_chars(text)
        text = re.sub(r'@[\w]*','',text)
        text = re.sub(r'http.*', ' ',text)
        #removes the @characters
        text = re.sub(r'[^a-zA-Z#0-9\s]',' ',text,re.I)
    #subsititute with space for anything other than the combination of [a-zA-Z0-9\s]. this is negated. PLease note tha
        text = text.translate(str.maketrans('','',string.punctuation))
        text = text.lower()
        text = text.strip()
        text = re.sub(r'gmailcom.*', ' ',text,re.I)
        text = re.sub(r'pictwittercom.*', ' ',text,re.I)
        text = re.sub(r'twittercom.*', ' ',text,re.I)
        
        
 #df1['post_title'].str.replace('http.*.com', '',regex = True)   - Source Stack Overflow    
    #Remove spaces at the beginning and at the end of the string:
        text = contractions.fix(text)
    #abbrevations for the words and are not slangs 
    #slangs like ttyl needs to be fixed. however these are abbrevations 
        tokens = nltk.word_tokenize(text)
        filtered_words = [token for token in tokens if token not in stop_words]
        text = " ".join(filtered_words)
        text = re.sub(' +',' ',text)
        norm_corpus.append(text)
    return norm_corpus
    

In [6]:
normalised_corpus = pre_process_corpus(combi)


  0%|                                                                                         | 0/9873 [00:00<?, ?it/s]C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 9873/9873 [00:02<00:00, 4800.20it/s]


In [11]:
normalised_corpus

['fingerprint pregnancy test',
 'finally transparant silicon case thanks uncle yay sony xperia sonyexperias',
 'love would go talk makememories unplug relax iphone smartphone wifi connect',
 'wired know george made way iphone cute daventry home',
 'amazing service apple even talk question unless pay 1995 stupid support',
 'iphone software update fucked phone big time stupid iphones',
 'happy us instapic instadaily us sony xperia xperiaz',
 'new type c charger cable uk',
 'bout go shopping listening music iphone justme music likeforlike followforfollow',
 'photo fun selfie pool water sony camera picoftheday sun instagood boy cute outdoor',
 'hey apple make new ipod make new color 2inches thinner make crash every five fuckin minite',
 'ha heavy machinery need really dropped ball design drinkyourhaterade',
 'contemplating giving iphone bandwagon simply cellcom new androids depressing idontwantto',
 'made another crazy purchase lol theory work hard play hard lol ipad apple shopping',
 'bat

In [16]:
train_normalised = normalised_corpus[:7920]

In [17]:
t = Tokenizer()
t.fit_on_texts(train_normalised)

In [18]:
vocab_size = len(t.word_index)+1
vocab_size

15758

In [25]:
encoded = t.texts_to_sequences(train_normalised)

In [29]:
#train 
encoded
# Node: 'sequential/embedding/embedding_lookup'
# indices[23,6] = 15938 is not in [0, 15758)
# 	 [[{{node sequential/embedding/embedding_lookup}}]] [Op:__inference_predict_function_30781]

[[1963, 5792, 1197],
 [39, 5793, 3761, 26, 61, 2342, 166, 6, 207, 5794],
 [8, 54, 76, 784, 5795, 3762, 393, 1, 160, 241, 1021],
 [1964, 93, 5796, 144, 223, 1, 21, 5797, 73],
 [45, 276, 2, 112, 784, 2887, 2888, 354, 5798, 161, 296],
 [1, 439, 50, 680, 5, 190, 28, 161, 334],
 [24, 133, 191, 145, 133, 6, 207, 1330],
 [4, 831, 596, 104, 424, 511],
 [1965, 76, 219, 951, 25, 1, 2343, 25, 750, 1702],
 [19, 32, 52, 1703, 440, 6, 68, 79, 168, 23, 181, 21, 2889],
 [146, 2, 44, 4, 36, 44, 4, 557, 5799, 2890, 44, 1022, 72, 1489, 1966, 5800],
 [1490, 2891, 5801, 98, 119, 785, 1104, 297, 5802],
 [3763, 708, 1, 5803, 1967, 5804, 4, 2344, 2892, 5805],
 [144, 150, 312, 1198, 78, 5806, 33, 228, 270, 228, 78, 13, 2, 219],
 [148, 3764, 298, 3765, 5807, 148, 641, 3766],
 [5808, 2893, 1968, 5809, 5810, 289, 5811, 6, 86, 1331],
 [9, 261, 56, 889, 5, 26, 1, 1],
 [76, 312, 66, 1, 23, 12, 42, 235],
 [340, 30, 1, 126, 2, 709, 5812, 952, 597, 3767, 335],
 [2, 127, 2894, 786, 1332, 341, 598, 71, 2, 127, 5813],
 [2

In [26]:
max_length = 38
padded_docs = pad_sequences(encoded,maxlen=max_length,padding='post')

In [20]:
t.word_index.items()

dict_items([('iphone', 1), ('apple', 2), ('samsung', 3), ('new', 4), ('phone', 5), ('sony', 6), ('follow', 7), ('love', 8), ('like', 9), ('day', 10), ('rt', 11), ('instagram', 12), ('ipad', 13), ('life', 14), ('gain', 15), ('got', 16), ('back', 17), ('get', 18), ('photo', 19), ('galaxy', 20), ('cute', 21), ('today', 22), ('instagood', 23), ('happy', 24), ('music', 25), ('case', 26), ('android', 27), ('time', 28), ('app', 29), ('one', 30), ('smile', 31), ('fun', 32), ('work', 33), ('ios', 34), ('twitter', 35), ('ipod', 36), ('beautiful', 37), ('itunes', 38), ('finally', 39), ('3', 40), ('everyone', 41), ('photooftheday', 42), ('good', 43), ('make', 44), ('amazing', 45), ('2', 46), ('must', 47), ('girl', 48), ('sougofollow', 49), ('update', 50), ('iphonex', 51), ('selfie', 52), ('followers', 53), ('would', 54), ('birthday', 55), ('want', 56), ('photography', 57), ('mac', 58), ('rts', 59), ('fuck', 60), ('thanks', 61), ('hate', 62), ('baby', 63), ('family', 64), ('pic', 65), ('iphonesia',

In [19]:
f = open(r"C:\Users\Thiyaneshwaran\glove.6B.100d.txt",encoding="utf8")
embedding_index = dict()
for line in f:
    values = line.split()
    #spitting by spaces 
    word = values[0]
    #1st value is the word 
    coefs = np.asarray(values[1:],dtype='float32')
    #coefficient are vaues from 1 to end and of type float
    embedding_index[word] = coefs
f.close()

In [22]:
embedding_matrix = np.zeros((vocab_size,100))

In [23]:
for word, i in t.word_index.items():
    embedding_vector = embedding_index.get(word)
    #print(embedding_index.get(word))
    if embedding_vector is not None:
        #print(i)
        embedding_matrix[i]=embedding_vector
        

In [24]:
target = train.label

In [27]:
model = Sequential()
e = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=38,trainable = False)
model.add(e)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 38, 100)           1575800   
                                                                 
 conv1d (Conv1D)             (None, 34, 128)           64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 17, 128)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2176)              0         
                                                                 
 dense (Dense)               (None, 1)                 2177      
                                                                 
Total params: 1,642,105
Trainable params: 66,305
Non-trainable params: 1,575,800
_________________________________________

In [30]:
model.fit(padded_docs,target,epochs=30,verbose=1)

Epoch 1/30
248/248 [==============================] - 2s 6ms/step - loss: 0.3587 - accuracy: 0.8388
Epoch 2/30
248/248 [==============================] - 2s 6ms/step - loss: 0.2643 - accuracy: 0.8845
Epoch 3/30
248/248 [==============================] - 2s 7ms/step - loss: 0.2116 - accuracy: 0.9138
Epoch 4/30
248/248 [==============================] - 2s 6ms/step - loss: 0.1596 - accuracy: 0.9423
Epoch 5/30
248/248 [==============================] - 2s 6ms/step - loss: 0.1150 - accuracy: 0.9641
Epoch 6/30
248/248 [==============================] - 2s 6ms/step - loss: 0.0808 - accuracy: 0.9789
Epoch 7/30
248/248 [==============================] - 2s 6ms/step - loss: 0.0536 - accuracy: 0.9896
Epoch 8/30
248/248 [==============================] - 2s 6ms/step - loss: 0.0387 - accuracy: 0.9937
Epoch 9/30
248/248 [==============================] - 2s 7ms/step - loss: 0.0277 - accuracy: 0.9957
Epoch 10/30
248/248 [==============================] - 2s 6ms/step - loss: 0.0197 - accuracy: 0.9981

In [31]:
train['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [32]:
test_normalised = normalised_corpus[7920:]

In [33]:
t.fit_on_texts(test_normalised)

In [34]:
encoded = t.texts_to_sequences(test_normalised)

In [36]:
max_length = 38
test_padded_docs = pad_sequences(encoded,maxlen=max_length,padding='post')

In [38]:
test_padded_docs

array([[   62,     4,     1, ...,     0,     0,     0],
       [ 1291,  6566,    66, ...,     0,     0,     0],
       [    9,  4072,  1187, ...,     0,     0,     0],
       ...,
       [  597,  1202,   130, ...,     0,     0,     0],
       [  556,  1870,   162, ...,     0,     0,     0],
       [   38,    17, 18107, ...,     0,     0,     0]])

In [37]:
y_Pred = model.predict(test_padded_docs)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Thiyaneshwaran\anaconda3\envs\deeplearning\lib\site-packages\tornado\platform\asyncio

In [ ]:
#Using the entire GLove Embedding 


In [39]:
f = open(r"C:\Users\Thiyaneshwaran\glove.6B.100d.txt",encoding="utf8")
embedding_index = dict()
for line in f:
    values = line.split()
    #spitting by spaces 
    word = values[0]
    #1st value is the word 
    coefs = np.asarray(values[1:],dtype='float32')
    #coefficient are vaues from 1 to end and of type float
    embedding_index[word] = coefs
f.close()

In [40]:
vocab_size_new = embedding_index.keys()
embedding_matrix_Values = embedding_index.values()
v = np.array(embedding_matrix_Values)

In [41]:
model = Sequential()
e = Embedding(400000,100,weights=v,input_length=38,trainable = False)
model.add(e)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

TypeError: len() of unsized object